In [12]:
# load dataset

import pandas as pd
from tabulate import tabulate
df = pd.read_csv('Employee Turnover Dataset.csv')
df.columns = df.columns.str.strip()
df.head()

,EmployeeNumber,Age,Tenure,Turnover,HourlyRate,HoursWeekly,CompensationType,AnnualSalary,DrivingCommuterDistance,JobRoleArea,Gender,MaritalStatus,NumCompaniesPreviouslyWorked,AnnualProfessionalDevHrs,PaycheckMethod,TextMessageOptIn
0,1,28,6,Yes,$24.37,40,Salary,50689.6,89,Research,Female,Married,3.0,7.0,Mail Check,Yes
1,2,33,2,Yes,$24.37,40,Salary,50689.6,89,Research,Female,Married,6.0,7.0,Mail Check,Yes
2,3,22,1,No,$22.52,40,Salary,46841.6,35,Information_Technology,Female,Single,1.0,8.0,Mailed Check,Yes
3,4,23,1,No,$22.52,40,Salary,46841.6,35,Information_Technology,Female,Single,3.0,NaN,Mailed Check,Yes
4,5,40,6,No,$88.77,40,Salary,284641.6,12,Sales,Prefer Not to Answer,Single,7.0,NaN,Mail Check,Yes


In [13]:
# dataset dimensions

total_rows, total_columns = df.shape
print(f"Total Records: {total_rows}")
print(f"Total Columns: {total_columns}")

Total Records: 10199
Total Columns: 16


In [14]:
# variable classification

def classify_variable(col):
    if pd.api.types.is_numeric_dtype(col):
        return "Quantitative", "Continuous" if col.nunique() > 15 else "Discrete"
    else:
        return "Qualitative", "Ordinal" if "status" in col.name.lower() else "Nominal"

profile = []
for col_name in df.columns:
    var_type, subtype = classify_variable(df[col_name])
    sample = df[col_name].dropna().unique()[:3].tolist()
    profile.append([col_name, var_type, subtype, sample])

profile_df = pd.DataFrame(profile, columns=["Variable", "Data Type", "Subtype", "Sample Values"])
print(tabulate(profile_df, headers='keys', tablefmt='grid', showindex=False))

+------------------------------+--------------+------------+--------------------------------------------------+
| Variable                     | Data Type    | Subtype    | Sample Values                                    |
+==============================+==============+============+==================================================+
| EmployeeNumber               | Quantitative | Continuous | [1, 2, 3]                                        |
+------------------------------+--------------+------------+--------------------------------------------------+
| Age                          | Quantitative | Continuous | [28, 33, 22]                                     |
+------------------------------+--------------+------------+--------------------------------------------------+
| Tenure                       | Quantitative | Continuous | [6, 2, 1]                                        |
+------------------------------+--------------+------------+--------------------------------------------

In [15]:
# duplicate entries

duplicates = df[df.duplicated()]
print(f"Duplicate Records Found: {len(duplicates)}")
if not duplicates.empty:
    display(duplicates)

Duplicate Records Found: 99


,EmployeeNumber,Age,Tenure,Turnover,HourlyRate,HoursWeekly,CompensationType,AnnualSalary,DrivingCommuterDistance,JobRoleArea,Gender,MaritalStatus,NumCompaniesPreviouslyWorked,AnnualProfessionalDevHrs,PaycheckMethod,TextMessageOptIn
10100,1,28,6,Yes,$24.37,40,Salary,50689.6,89,Research,Female,Married,3.0,7.0,Mail Check,Yes
10101,2,33,2,Yes,$24.37,40,Salary,50689.6,89,Research,Female,Married,6.0,7.0,Mail Check,Yes
10102,3,22,1,No,$22.52,40,Salary,46841.6,35,Information_Technology,Female,Single,1.0,8.0,Mailed Check,Yes
10103,4,23,1,No,$22.52,40,Salary,46841.6,35,Information_Technology,Female,Single,3.0,NaN,Mailed Check,Yes
10104,5,40,6,No,$88.77,40,Salary,284641.6,12,Sales,Prefer Not to Answer,Single,7.0,NaN,Mail Check,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10194,95,48,13,Yes,$85.40,40,Salary,177632.0,31,Research,Male,Single,7.0,5.0,Mail Check,NaN
10195,96,54,17,No,$85.40,40,Salary,177632.0,31,Research,Male,Single,2.0,25.0,Mail Check,Yes
10196,97,44,6,No,$71.90,40,Salary,149552.0,32,Marketing,Male,Married,6.0,NaN,Mail Check,Yes
10197,98,58,19,No,$71.90,40,Salary,149552.0,32,Marketing,Male,Married,5.0,23.0,Mail Check,Yes


In [16]:
# missing values

missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]
print("Missing Values:")
print(missing_values)

Missing Values:
NumCompaniesPreviouslyWorked     665
AnnualProfessionalDevHrs        1969
TextMessageOptIn                2266
dtype: int64


In [17]:
# inconsistent entries

import re
import pandas as pd

print("\nInconsistent Entries (key columns):")

def norm_key(s: str) -> str:
    # remove spaces and underscores
    return re.sub(r'[\s_]+', '', s.strip().lower())

# quick visual check
for col in ['PaycheckMethod', 'JobRoleArea']:
    if col not in df.columns:
        continue
    vals = df[col].dropna()
    original = sorted(vals.unique())
    normalized = sorted({norm_key(v) for v in vals})
    print(f"\n{col} – Original unique values:")
    for v in original:
        print(f"  '{v}'")
    print(f"After normalization (no spaces/underscores): {normalized}")


Inconsistent Entries (key columns):

PaycheckMethod – Original unique values:
  'Direct Deposit'
  'DirectDeposit'
  'Direct_Deposit'
  'Mail Check'
  'Mail_Check'
  'Mailed Check'
  'MailedCheck'
After normalization (no spaces/underscores): ['directdeposit', 'mailcheck', 'mailedcheck']

JobRoleArea – Original unique values:
  'Healthcare'
  'Human Resources'
  'HumanResources'
  'Human_Resources'
  'Information Technology'
  'InformationTechnology'
  'Information_Technology'
  'Laboratory'
  'Manufacturing'
  'Marketing'
  'Research'
  'Sales'
After normalization (no spaces/underscores): ['healthcare', 'humanresources', 'informationtechnology', 'laboratory', 'manufacturing', 'marketing', 'research', 'sales']


In [18]:
# standardize PaycheckMethod
if 'PaycheckMethod' in df.columns:
    pm_map = {
        'directdeposit': 'Direct Deposit',
        'mailedcheck': 'Mailed Check',
        'mailcheck': 'Mailed Check'  
    }
    # standardize/cleaning
    df['PaycheckMethod'] = df['PaycheckMethod'].apply(
        lambda x: pm_map.get(norm_key(x), re.sub(r'[_]+', ' ', x.strip()).title()) if pd.notna(x) else x
    )
    print("\nPaycheckMethod after standardization:")
    print(df['PaycheckMethod'].value_counts(dropna=False))
    print("Unique values:", sorted(df['PaycheckMethod'].dropna().unique()))

# standardize JobRoleArea
if 'JobRoleArea' in df.columns:
    jra_map = {
        'humanresources': 'Human Resources',
        'informationtechnology': 'Information Technology',
        'healthcare': 'Healthcare',
        'laboratory': 'Laboratory',
        'manufacturing': 'Manufacturing',
        'marketing': 'Marketing',
        'research': 'Research',
        'sales': 'Sales',
    }
    # standardize/cleaning
    df['JobRoleArea'] = df['JobRoleArea'].apply(
        lambda x: jra_map.get(norm_key(x), re.sub(r'[_]+', ' ', x.strip()).title()) if pd.notna(x) else x
    )
    print("\nJobRoleArea after standardization:")
    print(df['JobRoleArea'].value_counts(dropna=False))
    print("Unique values:", sorted(df['JobRoleArea'].dropna().unique()))




PaycheckMethod after standardization:
PaycheckMethod
Mailed Check      8023
Direct Deposit    2176
Name: count, dtype: int64
Unique values: ['Direct Deposit', 'Mailed Check']

JobRoleArea after standardization:
JobRoleArea
Research                  2025
Sales                     2007
Marketing                 1105
Manufacturing             1039
Laboratory                1021
Healthcare                1008
Human Resources           1001
Information Technology     993
Name: count, dtype: int64
Unique values: ['Healthcare', 'Human Resources', 'Information Technology', 'Laboratory', 'Manufacturing', 'Marketing', 'Research', 'Sales']


In [19]:
# data accuracy check - AnnualSalary vs calculated salary
print("\nData Accuracy Check: AnnualSalary vs Calculated Salary")

# make HourlyRate numeric
df['HourlyRate_num'] = df['HourlyRate'].replace(r'[\$,]', '', regex=True).astype(float)

# calculate expected salary assuming 52 weeks/year
df['CalculatedSalary'] = df['HourlyRate_num'] * df['HoursWeekly'] * 52

# flag mismatches
df['SalaryDifference'] = df['AnnualSalary'] - df['CalculatedSalary']
tolerance = 1  # dollars
salary_mismatches = df[abs(df['SalaryDifference']) > tolerance]

print(f"Number of inconsistent salary entries: {len(salary_mismatches)}")
if not salary_mismatches.empty:
    print(salary_mismatches[['HourlyRate_num', 'HoursWeekly', 'AnnualSalary', 'CalculatedSalary', 'SalaryDifference']].head())

# replace incorrect AnnualSalary with calculated one
df.loc[abs(df['SalaryDifference']) > tolerance, 'AnnualSalary'] = df['CalculatedSalary']


Data Accuracy Check: AnnualSalary vs Calculated Salary
Number of inconsistent salary entries: 2149
    HourlyRate_num  HoursWeekly  AnnualSalary  CalculatedSalary  \
4            88.77           40      284641.6          184641.6   
5            88.77           40      284641.6          184641.6   
16           88.50           40      284080.0          184080.0   
17           88.50           40      284080.0          184080.0   
22           87.22           40      281417.6          181417.6   

    SalaryDifference  
4           100000.0  
5           100000.0  
16          100000.0  
17          100000.0  
22          100000.0  


In [20]:
# formatting issues

categorical_cols = df.select_dtypes(include='object').columns

print("\nFormatting Issues:")
if 'HourlyRate' in df.columns:
    print("\nHourlyRate (sample values):")
    print(df['HourlyRate'].dropna().unique()[:5])
for col in categorical_cols:
    if df[col].dropna().apply(lambda x: x != x.strip()).any():
        print(f"Column '{col}' has extra spaces in some entries.")


Formatting Issues:

HourlyRate (sample values):
['$24.37 ' '$22.52 ' '$88.77 ' '$28.43 ' '$21.87 ']
Column 'HourlyRate' has extra spaces in some entries.


In [21]:
# outliers

df['HourlyRate'] = df['HourlyRate'].replace(r'[\$,]', '', regex=True).astype(float)
numeric_cols = [
    'Age', 'Tenure', 'HourlyRate', 'HoursWeekly',
    'AnnualSalary', 'DrivingCommuterDistance',
    'NumCompaniesPreviouslyWorked', 'AnnualProfessionalDevHrs'
]
print("\nOutlier Check:")
for col in numeric_cols:
    if col in df.columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower) | (df[col] > upper)]
        print(f"\n{col} – Outliers: {len(outliers)}")
        if not outliers.empty:
            print("Sample outlier values:", outliers[col].unique()[:5])


Outlier Check:

Age – Outliers: 0

Tenure – Outliers: 0

HourlyRate – Outliers: 0

HoursWeekly – Outliers: 0

AnnualSalary – Outliers: 0

DrivingCommuterDistance – Outliers: 245
Sample outlier values: [ 910  950  250 -125 -275]

NumCompaniesPreviouslyWorked – Outliers: 0

AnnualProfessionalDevHrs – Outliers: 0


In [22]:
# Export cleaned dataset
df_cleaned = df.drop_duplicates()
df_cleaned = df_cleaned[df_cleaned['DrivingCommuterDistance'] >= 0]

df_cleaned.to_csv('D599Task1_cleaned.csv', index=False)
print("\nCleaned data exported as 'D599Task1_cleaned.csv'")


Cleaned data exported as 'D599Task1_cleaned.csv'
